In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [21]:
train=pd.read_csv("/content/drive/MyDrive/Minor Final/Dataset/Final_Dataset/cauli_local.csv")
train.head()

,Commodity,Date,Average
0,Cauli Local,6/16/2013,32.5
1,Cauli Local,6/17/2013,27.5
2,Cauli Local,6/18/2013,27.5
3,Cauli Local,6/19/2013,27.5
4,Cauli Local,6/20/2013,22.5


In [22]:
train['Date']  = train['Date'].apply(pd.to_datetime)

In [23]:
(train["Date"] - train.Date.shift(1))

0         NaT
1      1 days
2      1 days
3      1 days
4      1 days
        ...  
3607   1 days
3608   1 days
3609   1 days
3610   1 days
3611   1 days
Name: Date, Length: 3612, dtype: timedelta64[ns]

In [24]:
(train["Date"] - train.Date.shift(1)).value_counts()

1 days    3500
2 days      92
3 days      12
4 days       4
8 days       1
7 days       1
5 days       1
Name: Date, dtype: int64

In [25]:
train.isna().sum()

Commodity    0
Date         0
Average      0
dtype: int64

In [26]:
px.line(
    data_frame=train,
    x="Date",
    y="Average"
)

In [27]:
resampled_df = train.resample("7D", on ='Date' , ).sum()
px.line(
    data_frame=resampled_df,
#     x="Date",
    y="Average"
)

<ipython-input-27-9834976e19c2>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [28]:
resampled_df

,Average
Date,
2013-06-16,159.5
2013-06-23,97.5
2013-06-30,222.5
2013-07-07,195.0
2013-07-14,337.5
...,...
2023-08-27,668.0
2023-09-03,656.0
2023-09-10,681.0


In [29]:
from sklearn.model_selection import train_test_split

y_train=train['Average']
# Splitting 'y_train' into training and testing sets without shuffling
y_train, y_test = train_test_split(y_train, test_size=0.2, shuffle=False)

In [30]:
# Training size:
print("Train set has {} observations.".format(len(y_train)))
# Test size:
print("Test set has {} observations.".format(len(y_test)))

Train set has 2889 observations.
Test set has 723 observations.


In [31]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
y_train=scaler.fit_transform(y_train.values.reshape(-1,1))
y_test=scaler.transform(y_test.values.reshape(-1,1))

In [32]:
# Sequence Data Preparation
SEQUENCE_SIZE = 10

def to_sequences(seq_size, obs):
    x = []
    y = []
    for i in range(0 , len(obs) - seq_size):
        window = obs[i:i + seq_size]
        after_window = obs[i + seq_size]
        x.append(window)
        y.append(after_window)
    return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

# Using y_test only
x_train, y_train = to_sequences(SEQUENCE_SIZE, y_train)
x_test, y_test = to_sequences(SEQUENCE_SIZE, y_test)

# Setup data loaders for batch
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [33]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [34]:
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Note: using Google CoLab
Using device: cpu


In [35]:
# Model definition using Transformer
class TransformerModel(nn.Module):
    def __init__(self, input_dim=1, d_model=64, nhead=4, num_layers=2, dropout=0.2):
        super(TransformerModel, self).__init__()

        self.encoder = nn.Linear(input_dim, d_model )
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers,enable_nested_tensor=True)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.decoder(x[:, -1, :])
        return x

model = TransformerModel(
    input_dim = 1,
    d_model = 32,
    nhead = 16,
    num_layers = 2,
    dropout= 0.05,
).to(device)

In [36]:
model

TransformerModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.05, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Linear(in_features=32, out_features=1, bias=True)
)

In [37]:
from torch.optim.lr_scheduler import ConstantLR
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)
scheduler = ConstantLR(optimizer)
epochs = 100
early_stop_count = 0
min_val_loss = float('inf')
train_losses = []
validation_losses = []
for epoch in range(epochs):
    model.train()
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        train_losses.append(loss.item())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in test_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())

    val_loss = np.mean(val_losses)
    scheduler.step()
    validation_losses.append(val_loss)
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    # if early_stop_count >= 5:
    #     print("Early stopping!")
    #     break
    print(f"Epoch {epoch + 1}/{epochs},Train Loss {loss.item()} Validation Loss: {val_loss:.4f}")

Epoch 1/100,Train Loss 0.26356592774391174 Validation Loss: 0.2055
Epoch 2/100,Train Loss 0.20087271928787231 Validation Loss: 0.2241
Epoch 3/100,Train Loss 0.18023668229579926 Validation Loss: 0.2273
Epoch 4/100,Train Loss 0.26991820335388184 Validation Loss: 0.2041
Epoch 5/100,Train Loss 0.1611970216035843 Validation Loss: 0.1936
Epoch 6/100,Train Loss 0.3781448304653168 Validation Loss: 0.2172
Epoch 7/100,Train Loss 0.3340894877910614 Validation Loss: 0.2135
Epoch 8/100,Train Loss 0.20645786821842194 Validation Loss: 0.2141
Epoch 9/100,Train Loss 0.16608279943466187 Validation Loss: 0.2024
Epoch 10/100,Train Loss 0.18421053886413574 Validation Loss: 0.1956
Epoch 11/100,Train Loss 0.19915011525154114 Validation Loss: 0.2007
Epoch 12/100,Train Loss 0.15701206028461456 Validation Loss: 0.1958
Epoch 13/100,Train Loss 0.17746978998184204 Validation Loss: 0.1966
Epoch 14/100,Train Loss 0.2676810920238495 Validation Loss: 0.2002
Epoch 15/100,Train Loss 0.1626008152961731 Validation Loss: 0

In [ ]:
# px.line(
#     x = range(len(train_losses)),
#     y = train_losses,
# )

In [38]:
torch.save(model,"/content/drive/MyDrive/Minor Final/Transformer_Final model/Transforme_cauli.pth")

In [39]:
model1 = torch.load("/content/drive/MyDrive/Minor Final/Transformer_Final model/Transforme_cauli.pth")

In [41]:
criterion = nn.MSELoss()
model1.eval()
predictions = []
valid_loss = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.tolist())
        validation_loss = criterion(outputs, y_batch)
        valid_loss.append(validation_loss.item())

# Convert predictions to a 2D numpy array
predictions = np.array(predictions).reshape(-1, 1)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, predictions))

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)

# Calculate R^2 score
r2 = r2_score(y_test, predictions)

# Print scores
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

RMSE: 0.4417
MAE: 0.2824
MSE: 0.1951
R^2 Score: 0.8221


In [48]:
import torch
import numpy as np

# Assuming `model1` is your loaded TransformerModel instance
model1.eval()  # Set the model to evaluation mode

# Define your current_sequence for prediction
current_sequence = np.array([0.39232334 ,0.39774334, 0.40259689 ,0.40713298 ,0.41148993,0.39232334 ,0.39774334, 0.40259689 ,0.40713298 ,0.41148993], dtype=np.float32)  # Fill in with your data

# Define the sequence length (assuming each prediction uses the last `seq_length` elements)
seq_length = 10

future_steps = 100  # Number of future steps to forecast
predictions = []

with torch.no_grad():  # Turn off gradients for prediction
    for _ in range(future_steps):
        # Extract the last `seq_length` elements from the current sequence
        input_sequence = current_sequence[-seq_length:]

        # Convert `input_sequence` to a torch tensor and reshape
        input_sequence_tensor = torch.tensor(input_sequence, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)

        # Perform prediction with the model
        output = model1(input_sequence_tensor)

        # Get the last prediction (sequence-to-one model)
        next_prediction = output.squeeze().item()

        # Append the predicted value to the list of predictions
        predictions.append(next_prediction)

        # Update `current_sequence` for the next prediction
        current_sequence = np.append(current_sequence, next_prediction)

# Now `predictions` contains the predicted values for the future steps
print(predictions)

[0.31633299589157104, 0.2633376121520996, 0.23275379836559296, 0.2114221304655075, 0.19378259778022766, 0.17823345959186554, 0.164506196975708, 0.15175199508666992, 0.13890129327774048, 0.1258305162191391, 0.11146897077560425, 0.0974678099155426, 0.08398959040641785, 0.07106523960828781, 0.05812770128250122, 0.04460923373699188, 0.02997933328151703, 0.013372570276260376, -0.005970999598503113, -0.02851080894470215, -0.05481161177158356, -0.08508102595806122, -0.1175745278596878, -0.14923568069934845, -0.1774253100156784, -0.20250527560710907, -0.22555936872959137, -0.24672280251979828, -0.26736170053482056, -0.28757697343826294, -0.3084043860435486, -0.3311033248901367, -0.35701119899749756, -0.38831019401550293, -0.428377628326416, -0.4780212640762329, -0.5342538356781006, -0.5919764041900635, -0.6459321975708008, -0.6964250802993774, -0.7433841228485107, -0.792137861251831, -0.8445180654525757, -0.8983325958251953, -0.9515432119369507, -1.0085054636001587, -1.0717214345932007, -1.142